In [1]:
from rss.api import (byte_locations, compressed_zarr,
                     parse_ebcdic, parse_binary_header, read_trace_data)


In [3]:
segy_file = 'FORGE_78-32_iDASv3-P11_UTC190419001218.sgy'

In [6]:
ebcdic = parse_ebcdic(segy_file)
print(ebcdic)

C1 ------------------------------ JOB.INFO -------------------------------------
C2 CLIENT: University of Utah, FIELD: FORGE, WELL: 78-32                        
C3 FIELD ENG1: Thomas Coleman, FIELD ENG2: Taylor Martin                        
C4 PD: MSL; RDR: GL; SDR: ; RDR ELEV: 1679.29M; SDR ELEV: 0.00M                 
C5 ----------------------------- FILE.INFO -------------------------------------
C6 ORIGINAL FILE: FORGE 78-32_UTC_20190419_001218.186.tdms                      
C7 SEGY GENERATED USING SEIBERBE 3.4.0.1, SEIBERFE 2.4.0.0                      
C8                                                                              
C9 RECORDING FORMAT: SEG-Y Rev. 100                                             
C10                                                                             
C11 FIRST SAMPLE 0MSEC, IEEE32FP, SR: 500MS                                     
C12                                                                             
C13 UTC File Time stamp: 201

In [7]:
binary_header = parse_binary_header(segy_file)    
binary_header

{'sample_rate_ms': 0.5,
 'ns': 30000,
 'float_format': 5,
 'units': 'feet',
 'size_of_trace': 120240,
 'num_traces': 1088}

### The goal is to make one of these for the DAS/DTS data

```
def read_trace_data(
    segy_file,
    binary_header,
    byte_locations=byte_locations,
    scalco=None,
    sort_order="inline",
):
    sort_order = sort_order.lower()
    if sort_order not in ("inline", "crossline"):
        raise RuntimeError(
            f"{sort_order} not supported, sort order should be on of inline or crossline."
        )

    if sort_order == "inline":
        orthogonal_line = "crossline"
    else:
        orthogonal_line = "inline"

    filename = os.path.splitext(os.path.basename(segy_file))[0]

    folder = f"{filename}"
    if not os.path.exists(folder):
        os.makedirs(folder)

    with open(os.path.join(filename, "binary_header.json"), "w") as fp:
        fp.write(json.dumps(binary_header))

    trace_size = binary_header["size_of_trace"]

    line_coords = defaultdict(list)

    inlines = np.zeros(binary_header["num_traces"], dtype=int)
    crosslines = np.zeros(binary_header["num_traces"], dtype=int)

    cdpx = np.zeros(binary_header["num_traces"], dtype=int)
    cdpy = np.zeros(binary_header["num_traces"], dtype=int)

    with open(segy_file, "rb") as fp:
        fp.seek(3600)
        for trac in tqdm.tqdm(range(0, binary_header["num_traces"])):
            raw_bytes = fp.read(binary_header["size_of_trace"])

            hdr = parse_header(raw_bytes, scalco)
            trace = parse_trace(raw_bytes, binary_header["float_format"])

            # Dumbest possible impl
            line_number = hdr[sort_order]
            folder = os.path.join(filename, f"{sort_order}s", f"{line_number}")
            if not os.path.exists(folder):
                os.makedirs(folder)
            with open(os.path.join(folder, "traces.bin"), "ba") as gp:
                trace.tofile(gp)
            line_coords[hdr[sort_order]].append(hdr[orthogonal_line])

            # save all the inlines
            inlines[trac] = hdr["inline"]
            crosslines[trac] = hdr["crossline"]

            cdpx[trac] = hdr["cdpx"]
            cdpy[trac] = hdr["cdpy"]

    np.save(os.path.join(filename, "inlines.npy"), inlines)
    np.save(os.path.join(filename, "crosslines.npy"), crosslines)

    np.save(os.path.join(filename, "cdpx.npy"), cdpx)
    np.save(os.path.join(filename, "cdpy.npy"), cdpy)

    inlines = np.unique(inlines)
    crosslines = np.unique(crosslines)

    min_inline = inlines[inlines > 0].min()
    max_inline = inlines[inlines > 0].max()

    min_crossline = crosslines[crosslines > 0].min()
    max_crossline = crosslines[crosslines > 0].max()

    num_inlines = len(inlines)
    num_crosslines = len(crosslines)

    if sort_order == "inline":
        min_orth_line = min_crossline
        num_orth_lines = max_crossline - min_crossline + 1
    else:
        min_orth_line = min_inline
        num_orth_lines = max_inline - min_inline + 1

    # seems backwards, but the "crosslines" are in the "inline" image
    # i.e. the inline number is constant and vice versa:
    for key, val in line_coords.items():
        indx = np.array(val) - min_orth_line
        valid_crosslines = np.zeros(num_orth_lines, dtype=bool)
        valid_crosslines[indx] = 1

        output_file = os.path.join(
            filename, f"{sort_order}s", f"{key}", "index.bin"
        )
        with open(output_file, "ba") as gp:
            valid_crosslines.tofile(gp)
```
